In [1]:
# Librerias
import numpy as np
from IPython.display import HTML, display

In [2]:
# Funciones de apoyo para graficar
def tableHtmlNumber(dataArr, decimal):
    # Inputs:
    # dataArr: Informacion en un formato numpy array
    
    # Formato para el header
    header = "".join(['<th style="text-align:center; vertical-align:middle">{}</th>'.format(i+1) for i in range(dataArr.shape[1])])
    header = '<th style="text-align:center; vertical-align:middle"></th>' + header
    
    # formato para las filas
    allMat = list()
    for row in range(dataArr.shape[0]):
        # Se recorre cada elemento
        filas = list()
        for j in dataArr[row]:
            # Se redondea decimal
            j_t = np.round(j,decimal)
            filas.append("".join('<td style="text-align:center; vertical-align:middle">{}</td>'.format(j_t)))
        
        # Se agrega el primer contador y el identificador de la fila
        allMat.append("<tr><th>{}</th>{}</tr>".format(row+1, "".join(filas)))
    
    # Se crea la tabla
    table = '<table style="margin: 0 auto">{}<tr>{}</tr></table>'.format(header, "".join(allMat))

    return table


def tableHtmlArrow(dataArr, dictPolicy):
    # Inputs:
    # dataArr: Informacion en un formato numpy array

    # Se extrae la dimension de la matriz
    filas = dataArr.shape[1]
    columns = dataArr.shape[0]
    
    # Codigos de flechas en html
    dictArrows = {(-1,0):"&uarr;", (0,1):"&rarr;",(1,0):"&darr;",(0,-1):"&larr;"}
    
    # Formato para el header
    header = "".join(['<th style="text-align:center; vertical-align:middle">{}</th>'.format(i+1) for i in range(filas)])
    header = '<th style="text-align:center; vertical-align:middle"></th>' + header

    # Se recodifica con las acciones
    allMat = list()
    for i in range(filas):
        # Se recorren las filas de la matrix
        filas = list()
        for j in range(columns):
            # Se extrae la informacion del diccionario
            acciones = "".join([dictArrows[tuple(i)] for i in dictPolicy[(i,j)]])
            filas.append("".join('<td style="text-align:center; vertical-align:middle">{}</td>'.format(acciones)))

        # Se agrega el primer contador y el identificador de la fila
        allMat.append("<tr><th>{}</th>{}</tr>".format(i+1, "".join(filas)))

    # Se crea la tabla
    table = '<table style="margin: 0 auto">{}<tr>{}</tr></table>'.format(header, "".join(allMat))

    return table



## Solución analítica
$v = R + \gamma P v$ 

$v = (I - \gamma P)^{-1}R$

In [3]:
import numpy as np

# Reglas de proceso
ladoTab = 5
sizeTab = ladoTab*ladoTab
dsct = 0.9
probMov = 0.25 # Probabilidad de ir a otro recuadro

# Nodos especiales
nodosEsp = {"nodoA": {"esp_A":[0,1], "esp_A_next":[4,1], "retorno":10},
            "nodoB": {"esp_B":[0,3], "esp_B_next":[2,3], "retorno":5}}

# -----------------------------------------------------------------------------------------
def probNodo(i,j,ladoTab,probMov):
    nodos = {"Arriba": np.array([i-1,j]),
             "Derecha": np.array([i,j+1]),
             "Abajo": np.array([i+1,j]),
             "Izquierdo":np.array([i,j-1]),
             "Actual":np.array([i,j])}

    # Retorno del nodo i, j
    salTablero = -1 # Castigo si sale del tablero
    retorno = 0 
    
    # Calculo de probabilidades por nodo
    probsNodo = dict()
    totProb = 0
    for key in nodos.keys():
        # Verificar que no se escape del tablero
        if (nodos[key].min() >= 0) and (nodos[key].max() < ladoTab):
            probsNodo[key] = probMov
        else:
            probsNodo[key] = 0
            totProb += probMov
            retorno += probMov * salTablero

    # Se calculan las probabilidades
    probsNodo["Actual"] = totProb 
        
    return nodos, probsNodo, retorno

# -----------------------------------------------------------------------------------------
def posiTransicion(nodos, probsNodo, ladoTab, nodosEsp):
    # Posiciones        
    posicion = np.zeros((ladoTab,ladoTab)) # Se construye matriz con posiciones asociadas al nodo evaluado

    # Caso especial A
    nodA = nodosEsp["nodoA"]["esp_A"]
    nodA_next = nodosEsp["nodoA"]["esp_A_next"]

    if (nodA[0] == nodos["Actual"][0] and nodA[1] == nodos["Actual"][1]):
        posicion[nodA_next[0],nodA_next[1]] = 1  # Se mueve con prob 1 al nodo A'
        return posicion.flatten()
        
    # Caso especial B
    nodB = nodosEsp["nodoB"]["esp_B"]
    nodB_next = nodosEsp["nodoB"]["esp_B_next"]

    if (nodB[0] == nodos["Actual"][0] and nodB[1] == nodos["Actual"][1]):
        posicion[nodB_next[0],nodB_next[1]] = 1 # Se mueve con prob 1 al nodo B'
        return posicion.flatten()
    
    # Casos normales
    for key in nodos.keys():
        if (probsNodo[key]>0):
            i_temp = nodos[key][0]
            j_temp = nodos[key][1]
            
            # Se coloca las probabilidades en la matriz
            posicion[i_temp,j_temp] = probsNodo[key]

    # La matriz se vuelve flat
    return posicion.flatten()
        

# Figura 3.5

In [4]:
# Solucion analitica
# -----------------------------------------------------------------------------------------
# Paso 1. Construye la matriz de transicion P y retorno R
MatP = list()
MatRetorno = np.zeros((ladoTab,ladoTab))
for i in range(ladoTab):
    for j in range(ladoTab):
        nodos, probsNodo, retorno= probNodo(i, j, ladoTab,probMov)
        MatP.append(posiTransicion(nodos, probsNodo, ladoTab, nodosEsp))
        MatRetorno[i,j] = retorno

# ------------------------------------------------------
# Se actualiza el retorno para los nodos A y B
i_A = nodosEsp["nodoA"]["esp_A"][0]
j_A = nodosEsp["nodoA"]["esp_A"][1]
retA = nodosEsp["nodoA"]["retorno"]
MatRetorno[i_A, j_A] = retA

i_B = nodosEsp["nodoB"]["esp_B"][0]
j_B = nodosEsp["nodoB"]["esp_B"][1]
retB = nodosEsp["nodoB"]["retorno"]
MatRetorno[i_B, j_B] = retB

# -----------------------------------------------------------------------------------------
# Paso 2: Solucion algebraica
vectRetorno = MatRetorno.flatten()
MatA = np.identity(sizeTab) - dsct*np.array(MatP) 
solucion = np.linalg.solve(MatA, vectRetorno)
solucion = solucion.reshape(ladoTab, -1) # Formato de matriz

In [5]:
display(HTML(tableHtmlNumber(solucion,decimal=1)))

1,3.3,8.8,4.4,5.3,1.5
2,1.5,3.0,2.3,1.9,0.5
3,0.1,0.7,0.7,0.4,-0.4
4,-1.0,-0.4,-0.4,-0.6,-1.2
5,-1.9,-1.3,-1.2,-1.4,-2.0


In [6]:
# Solucion numerica
# -----------------------------------------------------------------------------------------
valFun = np.random.randint(10, size=(ladoTab, ladoTab))

acciones = [[-1,0],[0,1],[1,0],[0,-1]]
rewardIn = 0
rewardOut = -1
nodosEsp = {"nodoA": {"esp_A":[0,1], "esp_A_next":[4,1], "retorno":10},
            "nodoB": {"esp_B":[0,3], "esp_B_next":[2,3], "retorno":5}}

centinela = True
while centinela:
    # Se monitorea los valores de la funcion valor
    valFun_new = np.zeros((ladoTab, ladoTab))
    for i in range(ladoTab):
        for j in range(ladoTab):
            for accion in acciones:
                # ----------------------------------------------------------------
                # Se evalua si se esta e un nodo especial A o B
                if ([i,j]==nodosEsp["nodoA"]["esp_A"]):
                    # Actualizacion de la funcion valor
                    i_temp = nodosEsp["nodoA"]["esp_A_next"][0]
                    j_temp = nodosEsp["nodoA"]["esp_A_next"][1]                  
                    valFun_new[i, j] += probMov * (nodosEsp["nodoA"]["retorno"] + dsct * valFun[i_temp, j_temp])
                
                elif ([i,j]==nodosEsp["nodoB"]["esp_B"]):
                    # Actualizacion de la funcion valor
                    i_temp = nodosEsp["nodoB"]["esp_B_next"][0]
                    j_temp = nodosEsp["nodoB"]["esp_B_next"][1]                  
                    valFun_new[i, j] += probMov * (nodosEsp["nodoB"]["retorno"] + dsct * valFun[i_temp, j_temp])

                # ----------------------------------------------------------------
                else:
                    # En caso sea un caso normal
                    i_temp = i+accion[0]
                    j_temp = j+accion[1]
    
                    # Actualizacion de la funcion valor dentro del tablero
                    if (min(i_temp,j_temp) >= 0) and (max(i_temp,j_temp) < ladoTab):
                        valFun_new[i, j] += probMov * (rewardIn + dsct * valFun[i_temp, j_temp])
                    else:
                        # En este caso se vuelve a la misma posicion i, j
                        valFun_new[i, j] += probMov * (rewardOut + dsct * valFun[i, j])

    # ---------------------------------------------------------------------------
    # Se evalua si el algoritmo converge. La funcion valor no cambia
    if np.sum(np.abs(valFun - valFun_new)) < 1e-5:
        # En caso de que no cambie se para
        centinela = False
    valFun = valFun_new


In [7]:
display(HTML(tableHtmlNumber(valFun,decimal=1)))

1,3.3,8.8,4.4,5.3,1.5
2,1.5,3.0,2.3,1.9,0.5
3,0.1,0.7,0.7,0.4,-0.4
4,-1.0,-0.4,-0.4,-0.6,-1.2
5,-1.9,-1.3,-1.2,-1.4,-2.0


# Figura 3.8

In [8]:
# -----------------------------------------------------------------------------------------
# Figura 3.8
valFun = np.random.randint(10, size=(ladoTab, ladoTab))
dictPolicy = dict()

acciones = [[-1,0],[0,1],[1,0],[0,-1]]
rewardIn = 0
rewardOut = -1
nodosEsp = {"nodoA": {"esp_A":[0,1], "esp_A_next":[4,1], "retorno":10},
            "nodoB": {"esp_B":[0,3], "esp_B_next":[2,3], "retorno":5}}

centinela = True
while centinela:
    # Se monitorea los valores de la funcion valor
    valFun_new = np.zeros((ladoTab, ladoTab))
    for i in range(ladoTab):
        for j in range(ladoTab):
            allVal = list()
            for accion in acciones:
                # ----------------------------------------------------------------
                # Se evalua si se esta e un nodo especial A o B
                if ([i,j]==nodosEsp["nodoA"]["esp_A"]):
                    # Actualizacion de la funcion valor
                    i_temp = nodosEsp["nodoA"]["esp_A_next"][0]
                    j_temp = nodosEsp["nodoA"]["esp_A_next"][1]
                    valCalulo = (nodosEsp["nodoA"]["retorno"] + dsct * valFun[i_temp, j_temp])
                
                elif ([i,j]==nodosEsp["nodoB"]["esp_B"]):
                    # Actualizacion de la funcion valor
                    i_temp = nodosEsp["nodoB"]["esp_B_next"][0]
                    j_temp = nodosEsp["nodoB"]["esp_B_next"][1]                  
                    valCalulo = (nodosEsp["nodoB"]["retorno"] + dsct * valFun[i_temp, j_temp])

                # ----------------------------------------------------------------
                else:
                    # En caso sea un caso normal
                    i_temp = i+accion[0]
                    j_temp = j+accion[1]
    
                    # Actualizacion de la funcion valor dentro del tablero. Recompensa = 0
                    if (min(i_temp,j_temp) >= 0) and (max(i_temp,j_temp) < ladoTab):
                        valCalulo = (rewardIn + dsct * valFun[i_temp, j_temp])
                    else:
                        # En este caso se vuelve a la misma posicion i, j. Recompensa = -1
                        valCalulo = (rewardOut + dsct * valFun[i, j])
                
                # Se guarda todos los valores maximos de la funcion valor para cada accion
                allVal.append(valCalulo)

            # Se guardan las politicas asociadas a la mayor funcion valor
            selPolicy = np.argwhere(allVal == np.amax(allVal))
            booleanLst  = [True if i in selPolicy else False for i in range(len(acciones))]            
            dictPolicy[(i,j)] = [x for x, flag in zip(acciones, booleanLst) if flag]
            valFun_new[i, j] = np.max(allVal)
            
    # ---------------------------------------------------------------------------
    # Se evalua si el algoritmo converge. La funcion valor no cambia
    if np.sum(np.abs(valFun - valFun_new)) < 1e-10:
        # En caso de que no cambie se para
        centinela = False
    else:
        valFun = valFun_new


In [9]:
display(HTML(tableHtmlNumber(valFun,decimal=1)))

1,22.0,24.4,22.0,19.4,17.5
2,19.8,22.0,19.8,17.8,16.0
3,17.8,19.8,17.8,16.0,14.4
4,16.0,17.8,16.0,14.4,13.0
5,14.4,16.0,14.4,13.0,11.7


In [10]:
display(HTML(tableHtmlArrow(valFun, dictPolicy)))

1,→,↑→↓←,←,↑→↓←,←
2,↑→,↑,↑←,←,←
3,↑→,↑,↑←,↑←,↑←
4,↑→,↑,↑←,↑←,↑←
5,↑→,↑,↑←,↑←,↑←


In [11]:
# Otra manera de calcular las politicas optimas
valOpti = valFun_new.copy() # Se trabaja en el optimo
accionOpti = dict()

# Solo se hace una corrida, pues ya estoy en el optimo
for i in range(ladoTab):
    for j in range(ladoTab):
        allVal = list()
        for accion in acciones:
            # Regla de movimiento
            i_temp = i+accion[0]
            j_temp = j+accion[1]

            # -------------------------------------------------------------------
            # Calculo de retorno asociado a la accion
            # ----------------------------------------------------------------
            # Se evalua si se esta e un nodo especial A o B
            if ([i,j]==nodosEsp["nodoA"]["esp_A"]):
                # Actualizacion de la funcion valor
                i_temp = nodosEsp["nodoA"]["esp_A_next"][0]
                j_temp = nodosEsp["nodoA"]["esp_A_next"][1]
                valCalulo = (nodosEsp["nodoA"]["retorno"] + dsct * valOpti[i_temp, j_temp])
            
            elif ([i,j]==nodosEsp["nodoB"]["esp_B"]):
                # Actualizacion de la funcion valor
                i_temp = nodosEsp["nodoB"]["esp_B_next"][0]
                j_temp = nodosEsp["nodoB"]["esp_B_next"][1]                  
                valCalulo = (nodosEsp["nodoB"]["retorno"] + dsct * valOpti[i_temp, j_temp])
            
            # Casos normales. Actualizacion de la funcion valor dentro del tablero. Recompensa = 0
            if (min(i_temp,j_temp) >= 0) and (max(i_temp,j_temp) < ladoTab):
                valCalulo = (rewardIn + dsct * valOpti[i_temp, j_temp])
            else:
                # En este caso se vuelve a la misma posicion i, j. Recompensa = -1
                valCalulo = (rewardOut + dsct * valOpti[i, j])
            
            # Se guarda la recompensa de cada accion
            allVal.append(valCalulo)

        # Se extrae la accion o acciones con mas valor
        selPolicy = np.argwhere(allVal == np.amax(allVal))
        booleanLst  = [True if i in selPolicy else False for i in range(len(acciones))]            
        accionOpti[(i,j)] = [x for x, flag in zip(acciones, booleanLst) if flag]

display(HTML(tableHtmlArrow(valOpti, accionOpti)))

1,→,↑→↓←,←,↑→↓←,←
2,↑→,↑,↑←,←,←
3,↑→,↑,↑←,↑←,↑←
4,↑→,↑,↑←,↑←,↑←
5,↑→,↑,↑←,↑←,↑←
